In [ ]:
# 킬링 끝음절 리스트 추가(추후에 해도 될듯)
# 비속어, 한글자 입력 시 다시 입력할 수 있게 하기   -> 완료
# 첫 음절만 같게 하고 띄어쓰기  -> 완료
# 특수문자 입력 시 다시 입력    -> 완료
# 끝음절이 자음 또는 모음 하나만 있을 경우 다시 입력    -> 완료
# 두음법칙  -> 완료

In [ ]:
import requests
import json
import random
import re

# 우리말샘 사전 API
api = 'API 입력력'

# 킬링 끝음절 리스트 지정
ban_last_chars = ['녘', '읊', '튕', '듈', '잚', '앎', '늪', '흙', '맑', '쳑', '딩', '팅', '늬', '뺌', '섯', '뺌',
                  '켬', '쁨', '찧', '앓', '륨', '뜀', '찝', '짊', '얽', '젖', '픔', '륨', '슘', '듐', '츔' '썹']
# 비속어 리스트 지정
ban_words = ['바보', '멍청이', '병신', '시발', '시바', '빙시', '븅신', '씨발', '씨바', '존나', '느금마', '개새끼', '지랄']
# 사용자가 입력한 단어 리스트
used_word = []
# 비속어, 띄어쓰기 등으로 잘못된 형식의 입력 단어 리스트
said_w = []

# 첫 입력
query = input("단어를 입력하세요(종료를 원하면 '그만'을 입력해주세요.): ")
# 두 글자 이상으로 단어 최소 길이 제한
while len(query) <= 1:
    said_w.append(query)
    query = input("두 글자 이상의 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")
    
# 비속어 제한
if query in ban_words:
    said_w.append(query)
    query = input("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요.")

#띄어쓰기 제한
if query.find(' ') != -1:
    said_w.append(query)
    query = input("올바른 단어를 입력해주세요.")

# 사용자가 입력한 단어 리스트에 추가 
used_word.append(query)

while True:
    # 단어 검색 요청
    url = f"https://opendict.korean.go.kr/api/search?certkey_no=6828&key={api}&target_type=search&req_type=json&part=word&q={query[-1]}&sort=date&start=1&num=100&advanced=y&method=start&type1=word&pos=1"
    response = requests.get(url).text
    response = json.loads(response)

    # 이용할 단어 담을 리스트 생성
    word_list = []

    # word_list 리스트에 단어 추가가
    for i in range(100):
        try:
            a = response['channel']['item'][i]['word']
            if a not in used_word:
                word_list.append(a)
        except:
            pass
        
        # 킬링 끝음절, 하이픈(-) 포함 단어 필터링링
        filtered_words = [
            w for w in word_list
            if w[-1] not in ban_last_chars
            and '-' not in w
        ]

    # com이 더이상 이을 수 있는 단어가 없을 때
    if not word_list:
        print('더 이상 이어지는 단어가 없습니다. 당신의 승리!')
        break

    # 필터링 된 단어에서 랜덤으로 추출
    com = random.choice(filtered_words)

    # 사용자 입력 단어와 com의 단어 출력
    print('you:', query, '|', 'com:', com, '\n', '-'*20)

    # com이 사용한 단어 리스트에 추가가
    used_word.append(com)

    # 두 번째 이상의 입력/입력해야할 첫 글자 언급
    query = input(f"{com[-1]}로 시작하는 단어를 입력해주세요(종료를 원하면 '그만'을 입력해주세요.): ")
    
    # 사용자 의지로 게임 종료
    if query == '그만': 
        print('게임 종료!')
        break
    
    # 잘못된 단어 입력
    if query[0] != com[-1]: 
        print(f'{query} 은(는) {com[-1]}로 시작하지 않습니다. com의 승리!')
        break

    # 중복 입력 금지
    if query in used_word: 
        print(f'{query} 은(는) 이미 입력한 단어입니다. com의 승리!')
        break
    
    # # 잘못된 형식의 단어 2회 이상 입력 시 게임 종료
    # if query in said_w:
    #     print('com의 승리!')
    #     break
    
    # 단어 최소 길이 제한
    if len(query) <= 1: 
        said_w.append(query)
        query = input('두 글자 이상의 단어를 입력해주세요.')
        continue

    # 비속어 제한
    if query in ban_words:  
        said_w.append(query)
        query = input("비속어는 사용할 수 없습니다. 올바른 단어를 입력해주세요.")
        continue
    
    # 띄어쓰기 제한
    if query.find(' ') != -1:
        said_w.append(query)
        query = input("올바른 단어를 입력해주세요.")
        print(said_w)
        continue


    # 사용자 입력 단어 리스트에 추가
    used_word.append(query)


you: 고  | com: 부담 
 --------------------
['고 ', '담 ']
you: 담장 | com: 장화 
 --------------------
게임 종료!


## 우리말샘 API를 이용한 한국어 끝말잇기 게임
### [규칙]
1. 사용자의 입력으로 게임이 시작한다.
2. 비속어, 특수문자, 잘못된 형식의 단어의 입력이 2회가 되면 게임은 com의 승리로 끝난다.
  - __잘못된 형식의 단어__ 의 예시
    1. 빈칸이 포함된 경우: '단 어', ' 단어', '단어 ', ' '
    2. 알파벳이 포함된 경우: 'eks어', '단어a'
    3. 한 글자 단어: '길', '눈'
    4. 숫자가 포함된 경우: '단어1', '단2어', '3단어'
    5. 모음 또는 자음이 단독으로 사용된 경우: '단ㄴ어', '단ㅓ', '단ㅇㅓ'
3. __그만, 끝, 종료__ 의 입력으로 게임을 중단할 수 있다.
4. __2글자 이상__ 의 단어로만 게임이 진행된다.
5. 두음법칙은 제시되는 경우에만 사용이 가능하다.

In [ ]:
# 두음법칙 샘플 func
def word_principle(last):
    if last[-1] in word_principle_dict.keys():
        word = input(f"{last[-1]}({word_principle_dict.get(last[-1])})(으)로 시작하는 단어를 입력해주세요.")

In [ ]:
# 완성본

import requests
import json
import random
import string

# 우리말샘 사전 API 발급 후 '' 안에 입력 
API_KEY = 'API 입력' 

# 금지할 킬링 끝음절 리스트
BAN_LAST_CHARS = ['션', '뤼', '녘', '읊', '튕', '듈', '잚', '앎', '늪', '흙', '맑', '쳑', '딩', '팅', '늬', '뺌',
                  '섯', '켬', '쁨', '찧', '앓', '륨', '뜀', '찝', '짊', '얽', '젖', '픔', '슘', '듐', '츔', '썹']
# 금지할 비속어 리스트
BAN_WORDS = ['바보', '멍청이', '병신', '시발', '시바', '빙시', '븅신', '씨발', '씨바', '존나', '느금마', '개새끼', '지랄']
# 숫자, 특수문자 리스트
chars = list(string.digits + string.ascii_letters + string.punctuation)
# 자음, 모음 리스트
koreans = [
    # 자음
    'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ',
    'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ',
    # 모음
    'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ',
    'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ',
    'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ',
    'ㅡ', 'ㅢ', 'ㅣ'
]
# 두음법칙 딕셔너리
word_principle_dict = {
"라":"나","락":"낙","란":"난","랄":"날",
"람":"남","랍":"납","랏":"낫","랑":"낭",
"략":"약","량":"양","렁":"넝","려":"여",
"녀":"여","력":"역","녁":"역","련":"연",
"년":"연","렬":"열","렴":"염","념":"염",
"렵":"엽","령":"영","녕":"영","로":"노",
"록":"녹","론":"논","롤":"놀","롬":"놈",
"롭":"놉","롯":"놋","롱":"농","료":"요",
"뇨":"요","룡":"용","뇽":"용","루":"누",
"룩":"눅","룬":"눈","룰":"눌","룸":"눔",
"룻":"눗","룽":"눙","류":"유","뉴":"유",
"륙":"육","률":"율","르":"느","륵":"늑",
"른":"는","를":"늘","름":"늠","릅":"늡",
"릇":"늣","릉":"능","래":"내","랙":"낵",
"랜":"낸","랠":"낼","램":"냄","랩":"냅",
"랫":"냇","랭":"냉","례":"예","뢰":"뇌",
"리":"이","니":"이","린":"인","닌":"인",
"릴":"일","닐":"일","림":"임","님":"임",
"립":"입","닙":"입","릿":"잇","닛":"잇",
"링":"잉","닝":"잉"
}

# 사용한 단어 저장 리스트
used_word = []

# 입력값 이용 가능 여부 판별 함수
# 단어 길이, 비속어, 공백, 알파벳, 특수문자, 숫자, 단독 자음, 단독 모음의 제한
# 제한된 범위에 속하면 False와 해당 경고 문장을 return
def get_input_validity(word):
    if len(word) <= 1:
        return False, "단어가 너무 짧습니다."
    if word in BAN_WORDS:
        return False, "비속어는 사용할 수 없습니다."
    if ' ' in word:
        return False, "공백은 사용할 수 없습니다."
    if any(char in chars for char in word): # chars라는 리스트의 요소가 word에 포함
        return False, "알파벳, 숫자 또는 특수문자는 사용할 수 없습니다."
    if any(char in koreans for char in word):
        return False, "자음, 모음 단독 글자는 사용할 수 없습니다."
    return True, ""

# 사용 불가 언어 사용 횟수(횟수가 2가 되면 게임 종료)
invalid_count = 0

# 두음법칙 적용 함수
def word_principle(last):
    if last[-1] in word_principle_dict.keys():
        word = input(f"{last[-1]}({word_principle_dict.get(last[-1])})(으)로 시작하는 단어를 입력해주세요.")
    return word

# 올바른 입력을 받기 위한 함수
def get_valid_input(prompt):
    """올바른 단어 입력 받기. 잘못된 입력이 2회 이상이면 None 반환"""
    global invalid_count
    while True:
        word = input(prompt).strip()
        if word in ['그만', '끝', '종료']:
            return None
    
        valid, message = get_input_validity(word)   # get_input_validity 함수에서 True,False 값(valid)과 문장(message)

        if not valid:
            # 비속어, 짧은 단어, 공백 포함 등 2회 이상 게임 종료
            if message in ["단어가 너무 짧습니다.", "비속어는 사용할 수 없습니다.", "공백은 사용할 수 없습니다."]:
                invalid_count += 1
                if invalid_count >= 2:
                    print("잘못된 입력이 2회 이상입니다. com의 승리!")
                    return None
            
            prompt = f"입력 오류: {message} 다시 입력하세요:"
            continue
        return word

# com이 출력할 단어 추출 함수
def get_next_word(last_char):
    try:
        url = (
            f"https://opendict.korean.go.kr/api/search?"
            f"certkey_no=6828&key={API_KEY}&target_type=search"
            f"&req_type=json&part=word&q={last_char}&sort=date"
            f"&start=1&num=100&advanced=y&method=start&type1=word&pos=1"
        )
        response = requests.get(url).text
        data = json.loads(response)

        word_list = []
        for item in data.get("channel", {}).get("item", []):
            word = item.get("word")
            if word and word not in used_word and '-' not in word and word[-1] not in BAN_LAST_CHARS and len(word) > 1:
                word_list.append(word)

        return word_list
    except Exception:
        print("API 요청 오류가 발생했습니다. API 키를 확인해주세요.")
        return []

def main():
    query = get_valid_input("단어를 입력하세요: ")
    if query == None:
        print("게임 종료!")
        return
    
    used_word.append(query)

    while True:
        next_words = get_next_word(query[-1]) or get_next_word(word_principle_dict.get(query[-1]))
        if not next_words:
            print("더 이상 이어지는 단어가 없습니다. 당신의 승리!")
            break

        com = random.choice(next_words)
        print(f"you: {query} | com: {com}\n" + "-" * 20)
        used_word.append(com)

        # while문 루프 전부 돌지를 않아서 같은 값을 입력해도 게임이 안 끝나고 있음/ 따로 함수로 빼서 작성해야할듯
        if word_principle_dict.get(com[-1]) == None:
            query = get_valid_input(f"{com[-1]}(으)로 시작하는 단어를 입력하세요.:")
            # continue
        else:
            query = get_valid_input(f"{com[-1]} 또는 {word_principle_dict.get(com[-1])}(으)로 시작하는 단어를 입력하세요: ")
            # continue

        if query == None:
            print("게임 종료!")
            break

        if query in used_word:
            print(f"{query} 은(는) 이미 입력한 단어입니다. com의 승리!")
            break
        
        if query[0] != com[-1] and query[0] != word_principle_dict.get(com[-1]):
            print(f"{query} 은(는) {com[-1]}로 시작하지 않습니다. com의 승리!")
            break

        used_word.append(query)

main()

you: 확인 | com: 인고
--------------------
you: 고의 | com: 의보
--------------------
you: 보안 | com: 안녕
--------------------
you: 영원 | com: 원율
--------------------
you: 율무 | com: 무량
--------------------
you: 양치 | com: 치깅스
--------------------
you: 스위스 | com: 스나이퍼
--------------------
you: 퍼즐 | com: 즐풍
--------------------
you: 풍자 | com: 자인
--------------------
you: 인도 | com: 도암
--------------------
you: 암살 | com: 살로
--------------------
you: 노쇠 | com: 쇠게
--------------------
you: 게시 | com: 시디디
--------------------
you: 디자인 | com: 인화
--------------------
you: 화장 | com: 장산
--------------------
you: 산장 | com: 장양
--------------------
you: 양산 | com: 산수
--------------------
you: 수산 | com: 산전
--------------------
you: 전의 | com: 의식주통
--------------------
you: 통발 | com: 발암
--------------------
암살 은(는) 이미 입력한 단어입니다. com의 승리!


In [ ]:
# 수정전전

import requests
import json
import random
import string

# 우리말샘 사전 API 발급 후 '' 안에 입력 
API_KEY = 'API 입력력'

# 금지할 킬링 끝음절 리스트
BAN_LAST_CHARS = ['녘', '읊', '튕', '듈', '잚', '앎', '늪', '흙', '맑', '쳑', '딩', '팅', '늬', '뺌',
                  '섯', '켬', '쁨', '찧', '앓', '륨', '뜀', '찝', '짊', '얽', '젖', '픔', '슘', '듐', '츔', '썹']
# 금지할 비속어 리스트
BAN_WORDS = ['바보', '멍청이', '병신', '시발', '시바', '빙시', '븅신', '씨발', '씨바', '존나', '느금마', '개새끼', '지랄']
# 숫자, 특수문자 리스트
chars = list(string.digits + string.ascii_letters + string.punctuation)
# 자음, 모음 리스트
koreans = [
    # 자음
    'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ',
    'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ',
    # 모음
    'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ',
    'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ',
    'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ',
    'ㅡ', 'ㅢ', 'ㅣ'
]
# 두음법칙 딕셔너리
word_principle_dict = {
"라":"나","락":"낙","란":"난","랄":"날",
"람":"남","랍":"납","랏":"낫","랑":"낭",
"략":"약","량":"양","렁":"넝","려":"여",
"녀":"여","력":"역","녁":"역","련":"연",
"년":"연","렬":"열","렴":"염","념":"염",
"렵":"엽","령":"영","녕":"영","로":"노",
"록":"녹","론":"논","롤":"놀","롬":"놈",
"롭":"놉","롯":"놋","롱":"농","료":"요",
"뇨":"요","룡":"용","뇽":"용","루":"누",
"룩":"눅","룬":"눈","룰":"눌","룸":"눔",
"룻":"눗","룽":"눙","류":"유","뉴":"유",
"륙":"육","률":"율","르":"느","륵":"늑",
"른":"는","를":"늘","름":"늠","릅":"늡",
"릇":"늣","릉":"능","래":"내","랙":"낵",
"랜":"낸","랠":"낼","램":"냄","랩":"냅",
"랫":"냇","랭":"냉","례":"예","뢰":"뇌",
"리":"이","니":"이","린":"인","닌":"인",
"릴":"일","닐":"일","림":"임","님":"임",
"립":"입","닙":"입","릿":"잇","닛":"잇",
"링":"잉","닝":"잉"
}

# 사용한 단어 저장 리스트
used_word = []

# 입력값 이용 가능 여부 판별 함수
# 단어 길이, 비속어, 공백, 알파벳, 특수문자, 숫자, 단독 자음, 단독 모음의 제한
# 제한된 범위에 속하면 False와 해당 경고 문장을 return
def get_input_validity(word):
    if len(word) <= 1:
        return False, "단어가 너무 짧습니다."
    if word in BAN_WORDS:
        return False, "비속어는 사용할 수 없습니다."
    if ' ' in word:
        return False, "공백은 사용할 수 없습니다."
    if any(char in chars for char in word): # chars라는 리스트의 요소가 word에 포함
        return False, "알파벳, 숫자 또는 특수문자는 사용할 수 없습니다."
    if any(char in koreans for char in word):
        return False, "자음, 모음 단독 글자는 사용할 수 없습니다."
    return True, ""

# 사용 불가 언어 사용 횟수(횟수가 2가 되면 게임 종료)
invalid_count = 0

# 두음법칙 적용 함수
def word_principle(last):
    if last[-1] in word_principle_dict.keys():
        word = input(f"{last[-1]}({word_principle_dict.get(last[-1])})(으)로 시작하는 단어를 입력해주세요.")
    return word

# 올바른 입력을 받기 위한 함수
def get_valid_input(prompt):
    """올바른 단어 입력 받기. 잘못된 입력이 2회 이상이면 None 반환"""
    global invalid_count
    while True:
        word = input(prompt).strip()
        if word in ['그만', '끝', '종료']:
            return None
    
        valid, message = get_input_validity(word)   # get_input_validity 함수에서 True,False 값(valid)과 문장(message)

        if not valid:
            # 비속어, 짧은 단어, 공백 포함 등 2회 이상 게임 종료
            if message in ["단어가 너무 짧습니다.", "비속어는 사용할 수 없습니다.", "공백은 사용할 수 없습니다."]:
                invalid_count += 1
                if invalid_count >= 2:
                    print("잘못된 입력이 2회 이상입니다. com의 승리!")
                    return None
            
            prompt = f"입력 오류: {message} 다시 입력하세요:"
            continue
        return word

# com이 출력할 단어 추출 함수
def get_next_word(last_char):
    try:
        url = (
            f"https://opendict.korean.go.kr/api/search?"
            f"certkey_no=6828&key={API_KEY}&target_type=search"
            f"&req_type=json&part=word&q={last_char}&sort=date"
            f"&start=1&num=100&advanced=y&method=start&type1=word&pos=1"
        )
        response = requests.get(url).text
        data = json.loads(response)

        word_list = []
        for item in data.get("channel", {}).get("item", []):
            word = item.get("word")
            if word and word not in used_word and '-' not in word and word[-1] not in BAN_LAST_CHARS and len(word) > 1:
                word_list.append(word)

        return word_list
    except Exception:
        print("API 요청 오류가 발생했습니다. API 키를 확인해주세요.")
        return []

def main():
    query = get_valid_input("단어를 입력하세요: ")
    # if query == None or query in ['그만', '끝', '종료']:
    #     return
    
    used_word.append(query)

    while True:
        if query == None or query in ['그만', '끝', '종료']:
            print("게임 종료!")
            break

        next_words = get_next_word(query[-1]) or get_next_word(word_principle_dict.get(query[-1]))
        if not next_words:
            print("더 이상 이어지는 단어가 없습니다. 당신의 승리!")
            break

        com = random.choice(next_words)
        print(f"you: {query} | com: {com}\n" + "-" * 20)
        used_word.append(com)
        # while문 루프 전부 돌지를 않아서 같은 값을 입력해도 게임이 안 끝나고 있음/ 따로 함수로 빼서 작성해야할듯
        if word_principle_dict.get(com[-1]) == None:
            query = get_valid_input(f"{com[-1]}(으)로 시작하는 단어를 입력하세요.:")
            continue
        if word_principle_dict.get(com[-1]) != None:
            query = get_valid_input(f"{com[-1]} 또는 {word_principle_dict.get(com[-1])}(으)로 시작하는 단어를 입력하세요: ")
            continue
        
        if query[0] != com[-1] and query[0] != word_principle_dict.get(com[-1]):
            print(f"{query} 은(는) {com[-1]}로 시작하지 않습니다. com의 승리!")
            break
        
        if query in used_word:
            print(f"{query} 은(는) 이미 입력한 단어입니다. com의 승리!")
            break

        used_word.append(query)

main()